# Gradient Descent and Momentum

This notebook explores gradient descent optimization from scratch, visualizing how different parameters affect convergence. I also implement momentum to show how it helps navigate tricky loss landscapes.

**Topics covered:**
- Gradient descent implementation
- Effect of step size and loss surface shape on convergence
- Momentum-based optimization
- Comparison on the Beale function (a classic optimization test case)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, ticker

## Part 1: Basic Gradient Descent

I'll start with a simple quadratic function: $f(x_1, x_2) = \frac{1}{2}(x_1^2 + a \cdot x_2^2)$

The parameter $a$ controls how "stretched" the loss surface is. When $a$ is large, the surface is much steeper in the $x_2$ direction.

In [ ]:
#the function and its gradient
def f_quadratic(x: np.ndarray, a: float) -> float:
    return 0.5 * (x[0]**2 + a * x[1]**2)

def df_quadratic(x: np.ndarray, a: float) -> np.ndarray:
    #gradient is [x1, a*x2]
    return np.array([x[0], a * x[1]], dtype=float)

In [ ]:
def gradient_descent_quadratic(a: float, step_size: float, num_steps: int) -> list:
    """Runs gradient descent on the quadratic function, starting from (256, 1)."""
    x = np.array([256.0, 1.0])
    path = [x.copy()]
    
    for _ in range(num_steps):
        g = df_quadratic(x, a)
        x = x - step_size * g
        path.append(x.copy())
    
    return path

In [ ]:
#this function visualizes the optimization path on a contour plot
def visualize_quadratic(a: float, path: list, ax=None) -> None:
    y_range = 10
    x = np.arange(-257, 257, 0.1)
    y = np.arange(-y_range, y_range, 0.1)
    xx, yy = np.meshgrid(x, y)
    z = 0.5 * (xx**2 + a * yy**2)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    ax.contourf(xx, yy, z, cmap='PuBu_r')
    ax.plot([p[0] for p in path], [p[1] for p in path], 'r.--', markersize=4, label='GD path')
    ax.plot([0], [0], 'rs', markersize=8, label='Optimum')
    ax.set_xlim([-257, 257])
    ax.set_ylim([-y_range, y_range])

### Effect of the parameter $a$ on convergence

When $a$ is small, the loss surface is nearly circular and gradient descent converges nicely. As $a$ increases, the surface becomes elongated and gradient descent starts to oscillate.

In [ ]:
#testing different values of a with fixed step size
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

a_values = [1, 4, 16, 64, 128, 256]

for ax, a in zip(axes.flatten(), a_values):
    #i had to reduce step size for larger a to prevent divergence
    step_size = 0.01 if a > 64 else 0.1
    path = gradient_descent_quadratic(a=a, step_size=step_size, num_steps=40)
    visualize_quadratic(a=a, path=path, ax=ax)
    ax.set_title(f'a={a}, step={step_size}')

plt.tight_layout()
plt.show()

**Observation:** Higher values of $a$ require smaller step sizes. This is because the gradient in the $x_2$ direction becomes much larger, causing overshooting if the step size is too big.

---

## Part 2: Gradient Descent with Momentum

To handle tricky loss landscapes, I implemented momentum. The idea is to accumulate a "velocity" that smooths out the optimization trajectory:

$$v_{t+1} = \beta v_t + (1-\beta) \nabla f(x_t)$$
$$x_{t+1} = x_t - \eta v_{t+1}$$

I'll test this on the Beale function, which has a nasty curved valley that makes vanilla gradient descent struggle.

In [ ]:
#the Beale function - a classic optimization benchmark
#it has a global minimum at (3, 0.5)
def f_beale(x: np.ndarray) -> float:
    return ((1.5 - x[0] + x[0]*x[1])**2 + 
            (2.25 - x[0] + x[0]*x[1]**2)**2 + 
            (2.625 - x[0] + x[0]*x[1]**3)**2)

def df_beale(x: np.ndarray) -> np.ndarray:
    #the gradient (provided in the exercise, this would be painful to derive by hand)
    d1 = (2*(1.5 - x[0] + x[0]*x[1])*(-1 + x[1]) + 
          2*(2.25 - x[0] + x[0]*x[1]**2)*(-1 + x[1]**2) + 
          2*(2.625 - x[0] + x[0]*x[1]**3)*(-1 + x[1]**3))
    d2 = (2*(1.5 - x[0] + x[0]*x[1])*(x[0]) + 
          2*(2.25 - x[0] + x[0]*x[1]**2)*(2*x[0]*x[1]) + 
          2*(2.625 - x[0] + x[0]*x[1]**3)*(3*x[0]*x[1]**2))
    return np.array([d1, d2])

In [ ]:
def gradient_descent(x0: np.ndarray, grad_fn, step_size: float, num_steps: int) -> list:
    """Vanilla gradient descent."""
    x = x0.copy()
    path = [x.copy()]
    
    for _ in range(num_steps):
        g = grad_fn(x)
        x = x - step_size * g
        path.append(x.copy())
    
    return path

def gradient_descent_momentum(x0: np.ndarray, grad_fn, step_size: float, num_steps: int, beta: float) -> list:
    """Gradient descent with momentum."""
    x = x0.copy()
    v = np.zeros_like(x)  #velocity starts at zero
    path = [x.copy()]
    
    for _ in range(num_steps):
        g = grad_fn(x)
        v = beta * v + (1 - beta) * g  #update velocity
        x = x - step_size * v          #update position
        path.append(x.copy())
    
    return path

In [ ]:
#this visualizes both paths on the Beale function
def visualize_beale(gd_path: list, momentum_path: list) -> None:
    x = np.arange(-4.5, 4.5, 0.01)
    y = np.arange(-4.5, 4.5, 0.01)
    xx, yy = np.meshgrid(x, y)
    z = f_beale(np.array([xx, yy]))
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.contourf(xx, yy, z, locator=ticker.LogLocator(), cmap='PuBu_r', levels=np.logspace(-2, 5, 35))
    
    ax.plot([p[0] for p in gd_path], [p[1] for p in gd_path], 'r.--', markersize=4, label='Vanilla GD')
    ax.plot([p[0] for p in momentum_path], [p[1] for p in momentum_path], 'm.--', markersize=4, label='With Momentum')
    ax.plot(3, 0.5, 'g*', markersize=15, label='Global minimum (3, 0.5)')
    
    ax.set_xlim([-4.5, 4.5])
    ax.set_ylim([-4.5, 4.5])
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.legend()
    ax.set_title('Beale Function: Gradient Descent vs Momentum')
    plt.show()

In [ ]:
#comparing vanilla GD vs momentum
start = np.array([-3.0, 3.0])

#vanilla GD needs tiny steps to not explode
gd_path = gradient_descent(start, df_beale, step_size=0.0001, num_steps=100)

#momentum can take bigger steps and still converge
momentum_path = gradient_descent_momentum(start, df_beale, step_size=0.001, num_steps=150, beta=0.95)

visualize_beale(gd_path, momentum_path)

print(f"Vanilla GD final position: {gd_path[-1]}")
print(f"Momentum final position: {momentum_path[-1]}")
print(f"Global minimum: (3, 0.5)")

## Results

The visualization shows how momentum helps:

1. **Vanilla GD (red)** barely moves - with such a small step size it would need thousands of iterations
2. **Momentum (magenta)** makes much more progress toward the minimum

The momentum term accumulates velocity in consistent directions while dampening oscillations. This is why optimizers like Adam (which uses momentum) are standard in deep learning - raw gradient descent is too slow and unstable for complex loss landscapes.